In [1]:
import pandas as pd
d = pd.read_csv('./data/rawBookInfo.csv', index_col=0)
ISBNs = d.iloc[:,1].tolist()

import dodoPipeline as dodo

result_1 = dodo.extractKyoboMultiThread(ISBNs[5400:])
print('result_1 완료')

kyoboBooks 추출완료
result_1 완료


In [2]:
asd = pd.DataFrame(result_1, columns=['book_title', 'book_toc', 'book_intro', 'publisher'])

asd.to_csv('./data/bookraw_10-10.csv')

### 수집하는 method 수정

In [ ]:
import requests
from bs4 import BeautifulSoup
import re 

ISBN = 9791158393359
kyoboUrl = f"http://www.kyobobook.co.kr/product/detailViewKor.laf?ejkGb=KOR&mallGb=KOR&barcode={ISBN}"
kyoboHtml = requests.get(kyoboUrl)
kyoboSoup = BeautifulSoup(kyoboHtml.content, "html.parser")

"""
book_title, book_toc, book_intro, publisher 추출하기

"""
book_title: str = kyoboSoup.find(class_="prod_title").string

# 도서 toc 추출, <br/>을 가지고 목차를 구분하므로 split 사용
book_toc: str = kyoboSoup.find(class_="book_contents_item")
book_toc = str(book_toc).split("<br/>")[1:-1]

# 도서 소개 추출, <br/>을 가지고 목차를 구분하므로 split 사용, tag제거
book_intro: str = kyoboSoup.find_all("div", "info_text")[-1]
book_intro_list = str(book_intro).split("<br/>")

book_intro = []
for i in book_intro_list:
    book_intro += re.sub("<.*>", "", i).split(".")

publisher: str = kyoboSoup.find(class_="book_publish_review")

if publisher is not None:
    publisher = publisher.p.get_text()
    publisher = publisher.split(".")
else:
    publisher = []

itemList = [book_title]

for chars in [book_toc, book_intro, publisher]:
    # ''제거
    chars = list(filter(None, chars))

    # 한국어 영어 제외하고 모두 제거
    chars = [
        re.sub("[^A-Za-z\u3130-\u318F\uAC00-\uD7A3]", " ", i) for i in chars
    ]

    # 공백 하나로 줄이기 ex) '  ' -> ' '
    chars = [re.sub(" +", " ", i).strip(" ") for i in chars]

    itemList.append(chars)

In [5]:
import re
re.sub("[^A-Za-z\u3130-\u318F\uAC00-\uD7A30-9.]", "", 'a0가나.▿▴') 

'a0가나.'